In [18]:
import json
from PyPDF2 import PdfReader
import re
import os

system_prompt = (
    "You are a landlord who is generating a lease for their tenants. "
    "You will be given a set of parameters, those parameters will fill in certain fields "
    "and those cannot be changed or you will be heavily penalized. Generate the residential lease agreement."
)

lease_params = [
    {
        "filename": "training_lease_one.pdf",
        "params": {
            "landlord_name": "ABC Properties",
            "tenant_name": "Silvia Mando",
            "property_address": "9876 Cherry Avenue, Apartment 426",
            "lease_start": "2012-07-01",
            "lease_end": "2013-06-30",
            "monthly_rent": 685,
            "security_deposit": 685,
            "state": "USA",
            "special_clauses": ["No smoking", "Minimum cleaning fee $200 on move-out"]
        }
    },
    {
        "filename": "training_lease_two.pdf",
        "params": {
            "landlord_name": "Elite Rentals LLC",
            "tenant_name": "Alex Johnson",
            "property_address": "456 Oak Street, Unit 12, Anytown, MD",
            "lease_start": "2024-05-01",
            "lease_end": "2025-04-30",
            "monthly_rent": 1800,
            "security_deposit": 1800,
            "state": "Maryland",
            "special_clauses": ["No smoking", "No pets"]
        }
    },
    {
        "filename": "training_lease_three.pdf",
        "params": {
            "landlord_name": "Carol Smith",
            "tenant_name": "Dana Lee",
            "property_address": "123 Maple Lane, Springfield, OH",
            "lease_start": "2025-06-01",
            "lease_end": "2025-07-01",  # placeholder for month-to-month
            "monthly_rent": 1200,
            "security_deposit": 1200,
            "state": "Ohio",
            "special_clauses": ["One-month notice required"]
        }
    },
    {
        "filename": "training_lease_four.pdf",
        "params": {
            "landlord_name": "Board of Regents of the Nevada System of Higher Education",
            "tenant_name": "John Doe Enterprises",
            "property_address": "Joe Crowley Student Union, University of Nevada, Reno Campus",
            "lease_start": "2016-01-01",
            "lease_end": "2021-12-31",
            "monthly_rent": 5000,
            "security_deposit": 5000,
            "state": "Nevada",
            "special_clauses": ["Tenant improvements per Exhibit B", "Option to extend per Exhibit G"]
        }
    }
]

def extract_and_fill(pdf_path, params):
    reader = PdfReader(pdf_path)
    text = "\n".join(page.extract_text() or "" for page in reader.pages)
    # Remove blank lines
    text = re.sub(r'\n{2,}', '\n', text)
    # Replace common placeholder patterns
    text = text.replace("___ day of ________", params["lease_start"])
    text = text.replace("beginning on ___ and ending on ___", f"beginning on {params['lease_start']} and ending on {params['lease_end']}")
    text = re.sub(r'\$\s*____+', f'${params["monthly_rent"]}', text)
    text = re.sub(r'Security Deposit.*?: \$.+?$', f"Security Deposit: ${params['security_deposit']}.", text, flags=re.MULTILINE)
    return text

output_path = "leases.jsonl"
with open(output_path, "w") as out_f:
    for lease in lease_params:
        full_path = os.path.join("", lease["filename"])
        completed_text = extract_and_fill(full_path, lease["params"])
        entry = {
            "prompt": system_prompt + "\n\n" + json.dumps(lease["params"]),
            "completion": completed_text
        }
        out_f.write(json.dumps(entry) + "\n")

output_path


'leases.jsonl'

In [14]:
import ollama
import json

lease_params = {
    "landlord_name":    "Alice Smith",
    "tenant_name":      "Bob Jones",
    "property_address": "123 Main St, Springfield, IL",
    "lease_start":      "2025-06-01",
    "lease_end":        "2026-05-31",
    "monthly_rent":     1200,
    "security_deposit": 1200,
    "state":            "Illinois",
    "special_clauses":  ["No smoking", "Pets allowed with $50/month pet rent"]
}

system_prompt = """
You are a legal drafting assistant. Generate a residential lease agreement that:
1. Uses the following parameters in appropriate lease-document format.
"""

param_summary = json.dumps(lease_params, indent=2)

response = ollama.generate(
    model="llama3.2",
    prompt=system_prompt + "\n\n" + param_summary,
    stream=False                 
)

full_text = response["response"] 
print(full_text)                 


ConnectionError: Failed to connect to Ollama. Please check that Ollama is downloaded, running and accessible. https://ollama.com/download